# **Procesamiento de Lenguaje Natural**

## Maestría en Inteligencia Artificial Aplicada
#### Tecnológico de Monterrey
#### Prof Luis Eduardo Falcón Morales

### **Adtividad en Equipos: sistema LLM + RAG**

* **Nombres y matrículas:**

  *   Carolina Gómez Fernández - A01305369
  *   Javier Alejandro Pérez Garza - A01284386
  *   Jorge Andrés Santos Gordon - A01652587
  *   Vanya Karime Betancourt Reyes - A01795070

* **Número de Equipo: 28**


* ##### **El formato de este cuaderno de Jupyter es libre, pero debe incuir al menos las siguientes secciones:**

  * ##### **Introducción de la problemática a resolver.**
  * ##### **Sistema RAG + LLM**
  * ##### **El chatbot, incluyendo ejemplos de prueba.**
  * ##### **Conclusiones**

* ##### **Pueden importar los paquetes o librerías que requieran.**

* ##### **Pueden incluir las celdas y líneas de código que deseen.**

In [ ]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...

!pip install langchain openai faiss-cpu tiktoken pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.2/304.2 kB 12.2 MB/s eta 0:00:00


In [ ]:
import os
os.environ["OPENAI_API_KEY"] = #'key' 

In [ ]:
!pip install langchain_community

from langchain.docstore.document import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

from google.colab import drive
drive.mount('/content/drive')

DIR = "/content/drive/MyDrive/Maestria/3 Trimestre/NLP/Semana 9 NLP"
os.chdir(DIR)

pdf_loader = PyPDFLoader('Enfermedades.pdf')
document = pdf_loader.load()

text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(document)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.3.44
    Uninstalling langsmith-0.3.44:
      Successfully uninstalled langsmith-0.3.44
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.63
    Uninstalling langchain-core-0.3.63:
      Successfully uninstalled langchain-core-0.3.63
Mounted at /content/drive


In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

embedding_model = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(docs, embedding_model)
retriever = vectorstore.as_retriever()

<ipython-input-4-1600479821>:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings()


In [ ]:
#import openai
#import os

# Initialize the OpenAI client.
# It will automatically pick up the OPENAI_API_KEY from the environment variables.
#client = openai.OpenAI()

# List available models
# Use the new syntax for listing models in openai>=1.0.0
#models = client.models.list()

# Print model IDs
#for model in models.data:  # Access the data attribute of the Models object
#    print(model.id)

In [ ]:
from langchain.prompts import PromptTemplate

custom_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
Eres un asistente, que responde preguntas sobre enfermedades usando únicamente el siguiente contexto.
Si no sabes la respuesta, solo dilo, no intentes inventar una.

Contexto:
{context}

Pregunta: {question}

Respuesta:"""
)

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": custom_prompt}
)

<ipython-input-7-3405649241>:4: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)


In [ ]:
query = "Qué es la apendicitis?"
response = rag_chain.run(query)
print(response)

<ipython-input-8-3020943038>:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = rag_chain.run(query)


La apendicitis es una inflamación del apéndice, que es una bolsa en forma de dedo que sobresale del colon en la parte inferior derecha del abdomen. Causa dolor en esa área, que generalmente comienza alrededor del ombligo y luego se desplaza hacia la parte inferior derecha del abdomen.


In [ ]:
!pip install gradio

In [ ]:
import gradio as gr

def responder_pregunta(pregunta):
    try:
        response = rag_chain.run(pregunta)
        return response
    except Exception as e:
        return f"Error: {str(e)}"

# Crear la interfaz
iface = gr.Interface(
    fn=responder_pregunta,
    inputs=gr.Textbox(lines=2, placeholder="Escriba su pregunta sobre Enfermedades"),
    outputs="text",
    title="Chatbot de Enfermedades",
    description="Ingrese una pregunta relacionada con alguna de estas enfermedades: Apendicitis, Anemia, Asma, Conjuntivitis, Covid, Diabetes, Gastritis, Gastroenteritis, Hipertensión Arterial, Infección Urinaria, Influenza, Migraña, Neumonia y el chatbot le responderá utilizando un modelo LLM y sistema RAG.",
)

# Lanzar la interfaz
iface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://41481a2b19148a668c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
